# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import requests
import collections
from bs4 import BeautifulSoup
import re
from nltk.util import ngrams

def addToArray(values,reference):
  for val in values:
    reference.append(val.text)
link=["https://www.imdb.com/title/tt7286456/reviews?ref_=tt_urv"]
rating=[]
text=[]
title=[]
for val in link:
  paper=requests.get(val,headers={'User-Agent':'Chrome/85.0.4183.121'})
  s = BeautifulSoup(paper.content, 'html.parser')
  addToArray(s.find_all('div', class_='text show-more__control'),text)
  addToArray(s.find_all('a',class_='title'),title)
  addToArray(s.find_all(name='span',class_='rating-other-user-rating'),rating)
imdbDataFrame=pd.DataFrame({'comment':text,'title':title})
imdbDataFrame.to_csv('imdb.csv')

def freqNgram(val,nB):
  dArray=[]
  for i in val:
    for j in  createNgrams(i, nB):
      dArray.append(j)
  freq=collections.Counter(dArray)
  return freq
threeGram=freqNgram(imdbDataFrame['comment'],3)
problem={}
oneFrame=freqNgram(imdbDataFrame['comment'],1)
twoFrame=freqNgram(imdbDataFrame['comment'],2)

def  createNgrams(val, nA):
  val = val.lower()
  val = re.sub(r'[^a-zA-Z0-9\s]', ' ', val)
  tokens = [token for token in val.split(" ") if token != ""]
  output = list(ngrams(tokens, nA))
  return output

for val in twoFrame.keys():
  problem[val]=twoFrame[val]/oneFrame[eval("'"+val[0]+"',")]

nounPhrase=[]
n_l_p = spacy.load("en_core_web_sm")
for val in imdbDataFrame['comment']:
  dox = n_l_p(val)
  for chunk in dox.noun_chunks:
    nounPhrase.append(chunk)
nounFreq=collections.Counter(nounPhrase)

nounPhrase[1:15]

big=max(nounFreq.values())
frequencyRelative=[]
for val in nounFreq.values():
  var=val/big
  frequencyRelative.append(var)
d={'noun_phrases':noun_phrases,'frequencyRelative':frequencyRelative}
frequencyRelative_df=pd.DataFrame(data=d)
frequencyRelative_df

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [ ]:
# Write your code here
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('punkt')

def TDM(dox, IDs=None):
    vect = CountVectorizer(lowercase=True, stop_words=None)
    tdm = vect.fit_transform(dox)
    tdm_feature_names = vect.get_feature_names()
    #
    dataFrame = pd.DataFrame(tdm.toarray(), columns=tdm_feature_names, dtype="float64")
    if IDs is not None:
        dataFrame.index = IDs    
    return dataFrame
TDM(imdbDataFrame['comment'])


stopWords = set(stopwords.words('english')) 
class TokenLemma:
    disregard = [':', '"', '``', "''", '`',',','.',';']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.disregard]
findTerm = 'movie'
tokenizer=TokenLemma()
token_stop = tokenizer(' '.join(stopWords))
documents = imdbDataFrame['comment']
v = TfidfVectorizer(stop_words=token_stop, 
                              tokenizer=tokenizer)
doc_vectors = v.fit_transform([findTerm] + documents)
cosSim = linear_kernel(doc_vectors[0:1], doc_vectors).flatten()
docScores = [item.item() for item in cosSim[1:]]

cosSim





# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

# Link: 
https://github.com/shivanigarlapati12/INFO-5731